In [ ]:
# Libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import random
from PIL import Image 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tensorflow.keras import Sequential, layers, Input, datasets, optimizers

In [ ]:
# Dataset
path = "C:/Users/Lucia/Desktop/statistical learning/final/raw-img"

translate = {
    "cane": "dog",
    "cavallo": "horse",
    "elefante": "elephant",
    "farfalla": "butterfly",
    "gallina": "chicken",
    "gatto": "cat",
    "mucca": "cow",
    "pecora": "sheep",
    "scoiattolo": "squirrel",
    "ragno": "spider"
}

data = {"imgpath": [], "labels": []}

for img_class in os.listdir(path):
     folderpath = os.path.join(path, img_class)
     filelist = os.listdir(folderpath)

     english_label = translate.get(img_class, img_class)  
     data['labels'] += [english_label] * len(filelist)
     data['imgpath'] += [os.path.join(folderpath, file) for file in filelist]

In [ ]:
# Dataset information
print("Number of images contaned in the dataset:\n", df.shape[0])
print("_______________________________________")
print("Number of null values:\n ", df.isnull().sum())
print("_______________________________________")
print("Number of unique values:\n", df.nunique())
print("_______________________________________")
counts = df['labels'].value_counts()
print("Number of images for every category:\n ", counts)

In [ ]:
# Data processing:
# Encode string labels to integers using LabelEncoder
label_encoder = LabelEncoder()
data["encoded_labels"] = label_encoder.fit_transform(data["labels"])

# Load images and convert them to NumPy arrays using TensorFlow
image_data = [tf.image.decode_jpeg(tf.io.read_file(img_path), channels=3) for img_path in data['imgpath']]
image_data = [tf.image.resize(img, (32, 32)) for img in image_data]
image_data = [tf.cast(img, tf.float32) / 255.0 for img in image_data]

# Ensure all images have the same dimensions
min_width = min(img.shape[0] for img in image_data)
min_height = min(img.shape[1] for img in image_data)
image_data = [tf.image.resize_with_crop_or_pad(img, min_width, min_height) for img in image_data]

# Convert labels to NumPy array
label_data = np.array(data["encoded_labels"])

# Splitting the dataset into train and test set
X_train, X_test, y_train, y_test = train_test_split(image_data, label_data, test_size=0.2, random_state=42)

# Transforming them into numpy array
tr_x = np.array(X_train)
tr_y = np.array(y_train)
te_x = np.array(X_test)
te_y = np.array(y_test)

def preprocess(x, y):
  x = tf.cast(x, tf.float32)/255.
  y = tf.cast(y, tf.int32)
  return x, y

tr_ds = tf.data.Dataset.from_tensor_slices((tr_x, tr_y))
te_ds = tf.data.Dataset.from_tensor_slices((te_x, te_y))

BATCH_SIZE = 128
tr_ds = tr_ds.map(preprocess).shuffle(len(tr_ds)).batch(batch_size = BATCH_SIZE)
te_ds = te_ds.map(preprocess).batch(batch_size = BATCH_SIZE)

In [ ]:
# ANN with dense layers

EPOCH = 100
LEARNING_RATE = 3e-4

# define the model
model_dense = Sequential([
  layers.InputLayer(input_shape=(32,32,3)),
  layers.Flatten(),
  layers.Dense(units=512, activation='relu'),
  layers.Dense(units=256, activation='relu'),
  layers.Dense(units=128, activation='relu'),
  layers.Dense(units=32, activation='relu'),
  layers.Dense(10)
])

model_dense.summary()

In [ ]:
# compile and fit the model
model_dense.compile(
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer = optimizers.Adam(learning_rate=LEARNING_RATE),
  metrics = ['accuracy']
)

history_dense = model_dense.fit (tr_ds, validation_data = te_ds, epochs = EPOCH,
verbose=2)

In [ ]:
# ANN with dropout
# define the model
model_dropout = Sequential([
  layers.InputLayer(input_shape=(32,32,3)),
  layers.Flatten(),
  layers.Dense(units=512, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(units=256, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(units=128, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(units=64, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(units=34, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(10)
])

model_dropout.summary()

In [ ]:
# compile and fit the model
model_dropout.compile(
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer = optimizers.Adam(learning_rate=LEARNING_RATE),
  metrics = ['accuracy']
)

history_dropout = model_dropout.fit (tr_ds, validation_data = te_ds, epochs =
EPOCH, verbose=2)

In [ ]:
# Losses and accuracies

# Dense model
acc_dense = history_dense.history["accuracy"]
val_acc_dense = history_dense.history["val_accuracy"]
loss_dense = history_dense.history["loss"]
val_loss_dense = history_dense.history["val_loss"]

# Dropout model
acc_dropout = history_dropout.history["accuracy"]
val_acc_dropout = history_dropout.history["val_accuracy"]
loss_dropout = history_dropout.history["loss"]
val_loss_dropout = history_dropout.history["val_loss"]

In [ ]:
#  Creating an ImageDataGenerator with data augmentation settings
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.3,
    height_shift_range=0.4
    shear_range=0.1
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode = "nearest")

In [ ]:
# Fit the data generator on the training data
datagen.fit(X_train)

# Creating the augmented training and test sets :
tr_aug = datagen.flow(tr_x, tr_y, batch_size=128)
te_aug = datagen.flow(te_x, te_y, batch_size=128)

In [ ]:
# Model
conv_model = Sequential([
    layers.InputLayer(input_shape=(32 ,32 ,3)),
    layers.Conv2D(64, kernel_size=3, padding="same"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(),
    layers.Conv2D(128, kernel_size=3, padding="valid"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(units=64, activation="relu"),
    layers.Dense(10, activation="sigmoid")
])

# Compile
conv_model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = optimizers.Adam(learning_rate=0.0001),
    metrics = ["accuracy"])

# Fit
conv_fit = conv_model.fit(tr_aug, validation_data = te_aug, epochs=EPOCH, batch_size=128)

In [ ]:
# Accuarcy and loss
acc_conv = conv_fit.history["accuracy"]
val_acc_conv = conv_fit.history["val_accuracy"]
loss_conv = conv_fit.history["loss"]
val_loss_conv = conv_fit.history["val_loss"]

In [ ]:
# figure 1
plt.figure(figsize=(6, 3.5))

plt.subplot(1, 2, 1)
plt.plot(range(EPOCH), acc_dense, label="Training Accuracy")
plt.plot(range(EPOCH), val_acc_dense, label="Validation Accuracy")
plt.legend(loc = "upper left")
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(range(EPOCH), loss_dense, label = "Training Loss")
plt.plot(range(EPOCH), val_loss_dense, label = "Validation Loss")
plt.legend(loc = "lower left")
plt.title("Loss")

plt.savefig("f1.png")

In [ ]:
# figure 2
plt.figure(figsize=(6, 3.5))

plt.subplot(1, 2, 1)
plt.plot(range(EPOCH), acc_dropout, label="Training Accuracy")
plt.plot(range(EPOCH), val_acc_dropout, label="Validation Accuracy")
plt.legend(loc = "upper left")
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(range(EPOCH), loss_dropout, label = "Training Loss")
plt.plot(range(EPOCH), val_loss_dropout, label = "Validation Loss")
plt.legend(loc = "lower left")
plt.title("Loss")

plt.savefig("f2.png")

In [ ]:
# figure 3
plt.figure(figsize=(6, 3.5))

plt.subplot(1, 2, 1)
plt.plot(range(EPOCH), acc_conv, label="Training Accuracy")
plt.plot(range(EPOCH), val_acc_conv, label="Validation Accuracy")
plt.legend(loc = "upper left")
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(range(EPOCH), loss_conv, label = "Training Loss")
plt.plot(range(EPOCH), val_loss_conv, label = "Validation Loss")
plt.legend(loc = "lower left")
plt.title("Loss")

plt.savefig("f3.png")

In [ ]:
# figure 4
sample = random.sample(range(len(data['imgpath'])), 5)

fig, axes = plt.subplots(1, 5, figsize=(15, 4))
for i, idx in enumerate(sample):
    img_path = data['imgpath'][idx]
    img = Image.open(img_path)
    axes[i].imshow(img)
    axes[i].set_title(f'Class: {data["labels"][idx]}')
    axes[i].axis('off')

plt.savefig('fB1.png')

In [ ]:
# figure 5
fig, axes = plt.subplots(1, 5, figsize=(15, 4))

for i, idx in enumerate(sample):
    img = image_data[idx].numpy() 
    label = label_encoder.inverse_transform([label_data[idx]])[0]
    
    axes[i].imshow(img)
    axes[i].set_title(f'Class: {label}')
    axes[i].axis('off')

plt.savefig('fB2.png')

In [ ]:
# figure 6
df = pd.DataFrame(data)

plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='labels', order=df['labels'].value_counts().index, palette='pastel')
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.xticks(rotation=45, ha='right')
plt.savefig('fB3.png')

In [ ]:
# figure 7
augmented_images = []
num_images_to_display = 5

for i in range(num_images_to_display):
    # Extraction of reconstructed images
    batch = next(tr_aug)
    augmented_image = batch[0][0]
    augmented_images.append(augmented_image)
    
    # Showing transformed images
    plt.figure(figsize=(10 , 5))
    for i in range(num_images_to_display):
        plt.subplot(1, num_images_to_display, i + 1)
        plt.imshow(augmented_images[i])
        plt.axis("off")
        
plt.show()